In [16]:
import pdfplumber
import pandas as pd
import re
from datetime import datetime
import numpy as np

In [22]:
class BankStatementParser:
    def __init__(self):
        self.date_patterns = [
            r'\d{2}/\d{2}/\d{4}',
            r'\d{2}-\d{2}-\d{4}',
            r'\d{2}\.\d{2}\.\d{4}'
        ]
        self.amount_pattern = r'[-+]?\d*,?\d+\.\d{2}'
        
    def extract_tables(self, pdf_path):
        """Extract all tables from PDF using pdfplumber"""
        with pdfplumber.open(pdf_path) as pdf:
            all_tables = []
            for page in pdf.pages:
                # Try different table settings
                tables = page.extract_tables(
                    table_settings={
                        "vertical_strategy": "text",
                        "horizontal_strategy": "text",
                        "intersection_y_tolerance": 10
                    }
                )
                if not tables:
                    # Try alternative settings if no tables found
                    tables = page.extract_tables(
                        table_settings={
                            "vertical_strategy": "lines",
                            "horizontal_strategy": "lines",
                            "intersection_y_tolerance": 5
                        }
                    )
                all_tables.extend(tables or [])
        return all_tables

    def clean_amount(self, amount_str):
        """Clean and convert amount string to float"""
        if not amount_str or pd.isna(amount_str):
            return np.nan
        
        # Remove currency symbols and spaces
        amount_str = re.sub(r'[£$€\s,]', '', str(amount_str))
        
        try:
            return float(amount_str)
        except ValueError:
            return np.nan

    def identify_columns(self, header_row):
        """Identify column types based on header content"""
        column_types = {}
        header_row = [str(h).lower() if h else '' for h in header_row]
        
        for idx, header in enumerate(header_row):
            if any(word in header for word in ['date', 'time', 'dt', 'dated', 'transaction date', 'value date', 'posting date']):
                column_types[idx] = 'date'
            elif any(word in header for word in ['desc', 'detail', 'narration', 'particular', 'description', 'transaction', 'remarks', 'details', 'narrative', 'ref', 'reference']):
                column_types[idx] = 'description'
            elif any(word in header for word in ['debit', 'withdrawal', 'paid', 'withdraw', 'dr', 'payments', 'amount paid', 'outgoing', 'sent']):
                column_types[idx] = 'debit'
            elif any(word in header for word in ['credit', 'deposit', 'received', 'cr', 'incoming', 'amount received', 'deposits', 'credited', 'receipt']):
                column_types[idx] = 'credit'
            elif any(word in header for word in ['balance', 'bal', 'closing balance', 'running balance', 'current balance', 'available balance']):
                column_types[idx] = 'balance'
            
        return column_types

    def process_row(self, row, column_types):
        """Process a single row according to identified column types"""
        processed_row = {'date': None, 'description': None, 'debit': 0.0, 'credit': 0.0, 'balance': None}
        
        try:
            for idx, value in enumerate(row):
                if idx not in column_types:
                    continue
                    
                col_type = column_types[idx]
                if not value:
                    continue
                    
                if col_type == 'date':
                    date_found = False
                    for pattern in self.date_patterns:
                        match = re.search(pattern, str(value))
                        if match:
                            try:
                                processed_row['date'] = datetime.strptime(match.group(), '%d/%m/%Y')
                                date_found = True
                                break
                            except ValueError:
                                continue
                    
                    if not date_found:
                        print(f"Warning: Could not parse date from value: {value}")
                                
                elif col_type == 'description':
                    processed_row['description'] = str(value).strip()
                    
                elif col_type in ['debit', 'credit', 'balance']:
                    amount_match = re.search(self.amount_pattern, str(value))
                    if amount_match:
                        amount = self.clean_amount(amount_match.group())
                        processed_row[col_type] = amount
                        
            return processed_row
        except Exception as e:
            print(f"Error processing row: {row}")
            print(f"Column types: {column_types}")
            raise e

    def tables_to_dataframe(self, tables):
        """Convert extracted tables to a structured DataFrame"""
        processed_rows = []
        
        try:
            for table_idx, table in enumerate(tables):
                if not table or len(table) < 2:  # Skip empty tables
                    print(f"Skipping table {table_idx}: Empty or too short")
                    continue
                    
                # Identify columns from header row
                column_types = self.identify_columns(table[0])
                if not column_types:  # Skip if no valid columns identified
                    print(f"Skipping table {table_idx}: No valid columns identified")
                    print(f"Header row: {table[0]}")
                    continue
                    
                print(f"Processing table {table_idx}")
                print(f"Identified columns: {column_types}")
                
                # Process each row
                for row_idx, row in enumerate(table[1:], 1):
                    processed_row = self.process_row(row, column_types)
                    if processed_row['date'] and (processed_row['debit'] or processed_row['credit']):
                        processed_rows.append(processed_row)
                    else:
                        print(f"Skipping row {row_idx} in table {table_idx}: Missing required fields")
                        print(f"Row content: {row}")
                        
            if not processed_rows:
                raise ValueError("No valid transactions found in the statement")
                
            # Create DataFrame
            df = pd.DataFrame(processed_rows)
            
            # Clean up
            df = df.dropna(subset=['date'])
            df = df.sort_values('date')
            df = df.reset_index(drop=True)
            
            return df
            
        except Exception as e:
            print(f"Error processing tables: {str(e)}")
            print(f"Number of tables found: {len(tables)}")
            raise e

    def parse_statement(self, pdf_path):
        """Main method to parse bank statement PDF"""
        try:
            tables = self.extract_tables(pdf_path)
            df = self.tables_to_dataframe(tables)
            
            # Add derived columns
            df['amount'] = df['credit'] - df['debit']
            df['transaction_type'] = df['amount'].apply(lambda x: 'CREDIT' if x > 0 else 'DEBIT')
            
            return df
            
        except Exception as e:
            print(f"Error parsing PDF {pdf_path}: {str(e)}")
            return None

In [23]:
if __name__ == "__main__":
    parser = BankStatementParser()
    
    # Add debugging information
    print("Attempting to parse PDF...")
    tables = parser.extract_tables("Bank Statements/AUB/205-2024   SOA2302234253810640bankau_bank.pdf")
    print(f"Found {len(tables)} tables")
    
    for idx, table in enumerate(tables):
        print(f"\nTable {idx}:")
        print(f"Number of rows: {len(table)}")
        if table:
            print(f"Header row: {table[0]}")
            print(f"First data row: {table[1] if len(table) > 1 else 'No data rows'}")
    
    # Try parsing
    df = parser.parse_statement("Bank Statements/AUB/205-2024   SOA2302234253810640bankau_bank.pdf")
    if df is not None:
        print("\nStatement parsed successfully:")
        print(df.head())
        print(f"\nTotal transactions: {len(df)}")

Attempting to parse PDF...
Found 4 tables

Table 0:
Number of rows: 110
Header row: ['', '', '', '', '', '', '', '', '', '', '', '']
First data row: ['', '', '', '', '', '', '', 'Account', 'Branch', ':', 'SV Road Mumbai', '']

Table 1:
Number of rows: 116
Header row: ['', '', '', '', '', '', '', '', '', '']
First data row: ['29/11/23', 'DRCRDTXN OTHBKFIN CHARGE FOR 28-N', 'OV-23', '29/11/23', '24.00', '', '', '1', '.00 24.00', '9,743.00']

Table 2:
Number of rows: 114
Header row: ['', '', '', '', '', '', '', '', '']
First data row: ['19/01/24', 'IMPS-401911113845-SOHANRAM-HDFC0000645-X', '19/01/24', '4,00,500.00', '', '', '1', '.00 4,00,500.00', '4,69,545.00']

Table 3:
Number of rows: 6
Header row: ['', '']
First data row: ['Cr', 'Count']
Skipping table 0: No valid columns identified
Header row: ['', '', '', '', '', '', '', '', '', '', '', '']
Skipping table 1: No valid columns identified
Header row: ['', '', '', '', '', '', '', '', '', '']
Skipping table 2: No valid columns identifie